In [1]:
import implicit
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender, bm25_weight, tfidf_weight

from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k
from implicit.evaluation import train_test_split

from matplotlib.pyplot import cm

from scipy.spatial.distance import cdist

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from implicit.als import AlternatingLeastSquares


from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

C:\Users\artemiyb1\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    try:
        recommended_list = recommended_list[:k]
    except:
        recommended_list = []
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [3]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

In [4]:
data_retail = pd.read_csv('retail_test.csv')

In [5]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [6]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [7]:
result_retail = data_retail.groupby('user_id')['item_id'].unique().reset_index()
result_retail.columns=['user_id', 'actual']
result_retail.head(2)

,user_id,actual
0,1,"[880007, 883616, 931136, 938004, 940947, 94726..."
1,2,"[820165, 820291, 826784, 826835, 829009, 85784..."


In [8]:
n_items_before = data_train['item_id'].nunique()

In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

C:\Users\artemiyb1\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [11]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [13]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [35]:
model = AlternatingLeastSquares(factors=4, 
                                regularization=0.05,
                                iterations=6, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),
          show_progress=True)

In [36]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  
                                    recalculate_user=True)]
    return res

In [37]:
result_test['bm25'] = result_test['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))

In [38]:
result_test.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()

0.1666013712046997

In [39]:
result_retail['bm25'] = result_retail['user_id'][result_retail['user_id']!=2325].apply(lambda x: get_recommendations(x, model=model, N=5))

In [40]:
result_retail.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()

<ipython-input-2-b6f6f2149963>:14: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


0.1522292993630561

In [41]:
result_retail

,user_id,actual,bm25
0,1,"[880007, 883616, 931136, 938004, 940947, 94726...","[1082185, 986947, 878996, 1127831, 854852]"
1,2,"[820165, 820291, 826784, 826835, 829009, 85784...","[1082185, 981760, 995785, 866211, 840361]"
2,3,"[827683, 908531, 989069, 1071377, 1080155, 109...","[1085604, 5568378, 883404, 1098066, 826249]"
3,6,"[956902, 960791, 1037863, 1119051, 1137688, 84...","[1082185, 995785, 1037863, 854852, 981760]"
4,7,"[847270, 855557, 859987, 863407, 895454, 90663...","[1082185, 6534178, 1029743, 826249, 995242]"
...,...,...,...
1880,2496,"[829291, 862139, 912704, 933067, 933835, 95537...","[1082185, 981760, 1098066, 826249, 1029743]"
1881,2497,[6534178],"[1082185, 981760, 995242, 840361, 923746]"
1882,2498,"[1053690, 1076875, 12386123, 858303, 920109, 1...","[1082185, 916122, 866211, 1127831, 961554]"
1883,2499,"[826249, 895327, 9858944, 820321, 829291, 8323...","[1098066, 826249, 883404, 1029743, 5568378]"


In [43]:
result_retail[['user_id', 'bm25']].to_csv('recommendations.csv', index=None)